# Training a CNN with the 256x256 images.

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm

from torchsummary import summary

# Import libraries for plot confusion matrix
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import itertools
import matplotlib.pyplot as plt

# All the figures will be 8,8
plt.rcParams['figure.figsize'] = [8, 8]

# Set the font to times and 18
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 18
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## First Baseline Model

In this notebook, I will be training a shallow CNN with just the right breast MLO view using 256x256 images.

### Dataset Object

The following Dataset object was created on purpose, to show what is the issue with the class unbalance on this kind of problems. Also, it is uploading ALL THE IMAGES, without caring about the modality of the image. 

In [ ]:
"""In this cell, I will be declaring the Dataset class, which will be used to load the images and the labels.
"""
df = pd.read_csv('../data/train.csv')
# df.sort_values(by='patient_id', inplace=True)
path_images = '../data/256_images/'
list_images = sorted(os.listdir(path_images))

# I will need a torch Dataset to load the images and the labels.
class DatasetBreastWrong(torch.utils.data.Dataset):
    """The labels will be available through df.cancer, whereas the images will be loaded from the /data/256_images folder.
    """
    def __init__(self, df, path_patients, transform=None):
        self.df = df
        self.path_patients = path_patients
        self.transform = transform
        self.list_patients = sorted(os.listdir(path_patients))
        self.list_patients = [x for x in self.list_patients if x != '.DS_Store']
        # From the list of patients, I will need to get access to the images
        self.list_images = []
        for patient in self.list_patients:
            for image in sorted(os.listdir(path_images + patient)):
                if image != '.DS_Store':
                    self.list_images.append(patient + '/' + image)
        self.list_images = sorted(self.list_images)
        self.labels = self.df.cancer.values
        self.labels = self.labels.astype(np.float32)
        self.labels = torch.from_numpy(self.labels)

    def __len__(self):
        return len(self.list_images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # Get the image
        path_image = self.path_patients + self.list_images[idx]
        image = plt.imread(path_image)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        # image = image.permute(2, 0, 1)
        # Get the label it should be either 0 or 1
        label = self.labels[idx]
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample
    

dataset_wrong = DatasetBreastWrong(df, path_images)
# print(dataset_wrong.list_images)
# Check if it is working
sample = dataset_wrong[5]
sample2 = dataset_wrong[20]
print(sample['image'].shape)
print(sample['label'])

# Show the image as a numpy array int32
plt.figure(figsize=(20, 10))
plt.subplot(1,2,1)
plt.imshow(sample['image'].numpy().astype(np.int32), cmap='gray')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(sample2['image'].numpy().astype(np.int32), cmap='gray')
plt.axis('off')
plt.show()


### Dataset with just one modality

In [ ]:
"""In this cell, I will create the proper Dataset object, using just one view like CC or MLO.

The dataset will consider the heavy inbalance of the data, so I will downsample the majority class.
"""
class DatasetBreastDownsample(torch.utils.data.Dataset):
    """The labels will be available through df.cancer, whereas the images will be loaded from the /data/256_images folder.
    """
    def __init__(self, df, path_patients, transform=None, view='CC', breast = 'L'):
        # From the dataframe, I will need to remove the images that are not the view I want. 
        # I will also need to remove the images that are not the breast I want.
        # Finally, I need to downsample the majority class. 
        self.df = df
        self.path_patients = path_patients
        self.transform = transform
        self.view = view
        self.breast = breast
         # Working with the dataframe
        self.df = self.df[self.df.view == self.view]
        self.df = self.df[self.df.laterality == self.breast]

        # I need to randomly select the patients that are in the majority class
        self.list_patients_majority = np.unique(self.df.patient_id[(self.df.cancer == 0).values])
        self.list_patients_minority = np.unique(self.df.patient_id[(self.df.cancer == 1).values])
        # I will need to downsample the majority class

        self.list_patients_majority = np.random.choice(self.list_patients_majority, 
                                                       size=len(self.list_patients_minority), 
                                                       replace=False)
        
        self.list_patients = np.concatenate((self.list_patients_majority, 
                                             self.list_patients_minority))
       
        # Reduce the dataframe to only have the list of patients
        self.df = self.df[self.df.patient_id.isin(self.list_patients)]
        self.df.reset_index(inplace=True)


        
        # From the df, I will need to construct the list of paths for the desired images, the format will be patient_id/image_id.dcm
        self.list_images = []
        for patient in self.list_patients:
            for image in self.df.image_id[self.df.patient_id == patient].values:
                self.list_images.append(str(patient) + '/' + str(image) + '.jpg')
        self.list_images = sorted(self.list_images)
        self.labels = self.df.cancer.values
        self.labels = self.labels.astype(np.float32)
        self.labels = torch.from_numpy(self.labels)

    def __len__(self):
        return len(self.list_images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # Get the image
        path_image = self.path_patients + self.list_images[idx]
        image = plt.imread(path_image)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        # image = image.permute(2, 0, 1)
        # Get the label it should be either 0 or 1
        label = self.labels[idx]
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample


dset_trial = DatasetBreastDownsample(df, path_images, view='CC', breast='L')
# Check if it is working
sample = dset_trial[0]
sample2 = dset_trial[20]
print(sample['image'].shape)
print(sample['label'])

# Show the image as a numpy array int32
plt.figure(figsize=(20, 10))
plt.subplot(1,2,1)
plt.imshow(sample['image'].numpy().astype(np.int32))
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(sample2['image'].numpy().astype(np.int32))
plt.axis('off')
plt.show()

In [ ]:

dset_trial.df

### Train-validation-test split

In [ ]:
# Train-validation-test split
train_size = int(0.8 * len(dset_trial))
val_size = int(0.1 * len(dset_trial))
test_size = len(dset_trial) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dset_trial, [train_size, val_size, test_size])


# Create the dataloaders
size_batch = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=size_batch, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=size_batch, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=size_batch, shuffle=True)

# Check if it is working
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(), sample_batched['label'])
    if i_batch == 3:
        break


Note that now we have a balanced dataset. With similar probabilities to find 1s and 0 labels. However, this is a very small set, which might be extremely hard to use for any Deep Learning Model. 

### Baseline Model (2D - Convolutional Neural Network)

In [ ]:
"""Create 2D convolutional neural network for the classification task of the cancer images (binary classification), based on images 3x256x256. I will be using a mini batch size of 32"""

# Create a model
model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, stride = 1, padding = 1))
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size = 2, stride = 2))
model.add_module('conv2', nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size = 2, stride = 2))
model.add_module('conv3', nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1))
model.add_module('relu3', nn.ReLU())
model.add_module('pool3', nn.MaxPool2d(kernel_size = 2, stride = 2))
# flatten
model.add_module('flatten', nn.Flatten())
model.add_module('fc1', nn.Linear(in_features = 32768, out_features = 512))
model.add_module('relu4', nn.ReLU())
model.add_module('dropout', nn.Dropout(p = 0.5))
model.add_module('fc2', nn.Linear(in_features = 512, out_features = 256))
model.add_module('relu5', nn.ReLU())
model.add_module('dropout', nn.Dropout(p = 0.5))
model.add_module('fc3', nn.Linear(in_features = 256, out_features = 1))
model.add_module('sigmoid', nn.Sigmoid())


model = model.to(device)
# Check if it is working
# sample = dataset[5]
# sample2 = dataset[20]
# print(sample['image'].shape)
# print(sample['label'])
# print(model(sample['image'].unsqueeze(0)).item())
# print(model(sample2['image'].unsqueeze(0)).item())

# Summary
summary(model, (3, 256, 256))

In [ ]:
# Train the model 

# Define the loss function
criterion = nn.BCELoss()
# Define a loss function with l2 regularization
# criterion = nn.BCELoss() + 0.01*torch.norm(model.fc1.weight, 2) + 0.01*torch.norm(model.fc2.weight, 2)

# Define the optimizer with l2 regularization
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, weight_decay=1, momentum=0.5)

# Train the model
def train(model, num_epochs, train_dl, valid_dl, track = False):
    loss_hist_train = [0]*num_epochs
    loss_hist_valid = [0]*num_epochs
    accuracy_hist_train = [0]*num_epochs
    accuracy_hist_valid = [0]*num_epochs
    for epoch in tqdm.tqdm(range(num_epochs)):
        model.train()
        for i, sample_batched in enumerate(train_dl):
            optimizer.zero_grad()
            outputs = model(sample_batched['image'])
            loss = criterion(outputs, sample_batched['label'].unsqueeze(1))
            loss.backward()
            optimizer.step()
            loss_hist_train[epoch] += loss.item()
            accuracy_hist_train[epoch] += (outputs.round() == sample_batched['label'].unsqueeze(1)).sum().item()
        loss_hist_train[epoch] /= len(train_dl)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)
        model.eval()
        with torch.no_grad():
            for i, sample_batched in enumerate(valid_dl):
                outputs = model(sample_batched['image'])
                loss = criterion(outputs, sample_batched['label'].unsqueeze(1))
                loss_hist_valid[epoch] += loss.item()
                accuracy_hist_valid[epoch] += (outputs.round() == sample_batched['label'].unsqueeze(1)).sum().item()
        loss_hist_valid[epoch] /= len(valid_dl)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)

        # print loss and accuracy for the train set and validation set
        if track:
            print('Epoch [{}/{}], train_Loss: {:.4f}, train_acc: {:.2f}%'.format(epoch+1, num_epochs, loss_hist_train[epoch], accuracy_hist_train[epoch]*100))
            print('val_loss: {:.4f}, val_acc: {:.2f}%'.format(loss_hist_valid[epoch], accuracy_hist_valid[epoch]*100))
    return model, loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid
num_epochs = 100

model, loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid = train(model, num_epochs, train_loader, val_loader, track = True)

# Plot the loss and accuracy for training
plt.plot(loss_hist_train, label='Training loss')
plt.plot(loss_hist_valid, label='Validation loss')
plt.legend(frameon=False)
plt.title("Loss")
plt.show()

# Plot the loss and accuracy for validation
plt.plot(accuracy_hist_train, label='Training accuracy')
plt.plot(accuracy_hist_valid, label='Validation accuracy')
plt.legend(frameon=False)
plt.title("Accuracy")
plt.show()




Note that the validation accuracy is extremely unstable, and my best model had an accuracy on the validation set of 60%. Note also that ther training accuracy and the validation accuracy are extremely different, which suggests overfitting. 

Below, I am presenting the test loss and accuracy. 

In [ ]:
# test set
model.eval()
with torch.no_grad():
    loss_hist_test = 0
    accuracy_hist_test = 0

    for i, sample_batched in enumerate(test_loader):
        outputs = model(sample_batched['image'])
        loss = criterion(outputs, sample_batched['label'].unsqueeze(1))
        loss_hist_test += loss.item()
        accuracy_hist_test += (outputs.round() == sample_batched['label'].unsqueeze(1)).sum().item()
    loss_hist_test /= len(test_loader)
    accuracy_hist_test /= len(test_loader.dataset)

    print('test_loss: {:.4f}, test_acc: {:.2f}%'.format(loss_hist_test, accuracy_hist_test*100))



In [ ]:
# Compute the confusion matrix
model.eval()
import seaborn as sns
with torch.no_grad():
    y_pred = []
    y_true = []
    for i, sample_batched in enumerate(test_loader):
        outputs = model(sample_batched['image'])
        y_pred.extend(outputs.round().tolist())
        y_true.extend(sample_batched['label'].tolist())
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(10,10))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title("Confusion matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig("../results/confusion_baseline.png")
    
    plt.show()                                                      
    

    
    
# Compute the ROC curve
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for i, sample_batched in enumerate(test_loader):
        outputs = model(sample_batched['image'])
        y_pred.extend(outputs.tolist())
        y_true.extend(sample_batched['label'].tolist())
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    print("AUC: {:.2f}".format(auc))
    plt.plot(fpr, tpr, label = f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title("ROC curve")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.legend()
    plt.savefig("../results/roc_baseline.png")

    plt.show()


    

In [ ]:
# Plot the loss and accuracy for training
plt.plot(loss_hist_train, label='Training loss')
plt.plot(loss_hist_valid, label='Validation loss')
plt.legend(frameon=False)
plt.title("Loss")
plt.savefig("../results/loss_baseline.jpg")
plt.xlabel('Epoch')
plt.show()

# Plot the loss and accuracy for validation
plt.plot(accuracy_hist_train, label='Training accuracy')
plt.plot(accuracy_hist_valid, label='Validation accuracy')
plt.legend(frameon=False)
plt.title("Accuracy")
plt.savefig("../results/accuracy_baseline.jpg")
plt.xlabel('Epoch')
plt.show()


In [ ]:
# save model
torch.save(model.state_dict(), "../results/baseline.pt")